In [9]:
fff=!pwd
print(fff[0]+"/analysis_expression3_for_inference.ipynb")

/workspace/02_cha_drug_repositioning/analysis_expression3_for_inference.ipynb


In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [11]:
import numpy as np
import pandas as pd
import copy
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
import pickle

from cmapPy.pandasGEXpress.parse import parse
from cmapPy.pandasGEXpress.write_gct import write
from cmapPy.pandasGEXpress.GCToo import GCToo
from cmapPy.pandasGEXpress.gctx2gct import gctx2gct_main

import torch

import warnings
warnings.filterwarnings('ignore')

In [12]:
from super_classes import param_class as param_class_module
from utils import data_processing as data_processing_module
from utils import data_load as data_load_module
from run import train as train_module
from run import test_model as test_model_module

# Configure parameters

In [13]:
BASE_DIR="/workspace/datafiles/2023/04/02_cha_drug_repositioning/paper_data"
GCTX_PATH_GSE70138="GSE70138_Broad_LINCS_Level4_ZSPCINF_mlr12k_n345976x12328_2017-03-06.gctx"
GCTX_PATH_GSE92742="GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx"
TXT_META_PATH_GSE70138="GSE70138_Broad_LINCS_inst_info_2017-03-06.txt"
TXT_META_PATH_GSE92742="GSE92742_Broad_LINCS_inst_info.txt"
PERT_META_PATH_GSE70138="GSE70138_Broad_LINCS_pert_info_2017-03-06.txt"
PERT_META_PATH_GSE92742="GSE92742_Broad_LINCS_pert_info.txt"
TXT_LANDMARK_GENES_PATH="GSE92742_Broad_LINCS_gene_info_delta_landmark.txt"

In [14]:
configurable_parameters_dict={
  "BASE_DIR":BASE_DIR,
  "GCTX_PATH_GSE70138":GCTX_PATH_GSE70138,
  "GCTX_PATH_GSE92742":GCTX_PATH_GSE92742,
  "TXT_META_PATH_GSE70138":TXT_META_PATH_GSE70138,
  "TXT_META_PATH_GSE92742":TXT_META_PATH_GSE92742,
  "PERT_META_PATH_GSE70138":PERT_META_PATH_GSE70138,
  "PERT_META_PATH_GSE92742":PERT_META_PATH_GSE92742
}

# Load data

In [15]:
loaded_gctx_GSE70138=data_load_module.Utils_Data_Load.load_gctx(BASE_DIR,GCTX_PATH_GSE70138)
loaded_gctx_GSE92742=data_load_module.Utils_Data_Load.load_gctx(BASE_DIR,GCTX_PATH_GSE92742)

In [8]:
loaded_gctx.head(2)

NameError: name 'loaded_gctx' is not defined

In [9]:
loaded_gctx.shape

(12328, 1319138)

# For fast test

In [ ]:
loaded_gctx_GSE70138=loaded_gctx_GSE70138.iloc[:,:100000]
loaded_gctx_GSE92742=loaded_gctx_GSE92742.iloc[:,:100000]

# Clean column and row names

In [16]:
loaded_gctx_GSE70138=data_processing_module.Utils_Data_Processing.clean_column_row_names(loaded_gctx_GSE70138)
loaded_gctx_GSE92742=data_processing_module.Utils_Data_Processing.clean_column_row_names(loaded_gctx_GSE92742)

In [11]:
display(loaded_gctx_GSE70138.head(2))
display(loaded_gctx_GSE92742.head(2))

,CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,CPC005_A375_6H_X3_B3_DUO52HI53LO:K06,CPC005_A375_6H_X1_B3_DUO52HI53LO:C19,CPC005_A375_6H_X2_B3_DUO52HI53LO:C19,CPC005_A375_6H_X3_B3_DUO52HI53LO:C19,CPC004_A375_6H_X1_B3_DUO52HI53LO:K13,CPC004_A375_6H_X2_B3_DUO52HI53LO:K13,CPC004_A375_6H_X3_B3_DUO52HI53LO:K13,CPC005_A375_6H_X1_B3_DUO52HI53LO:K20,...,PCLB003_PC3_24H_X3_B13:P15,PCLB003_PC3_24H_X3_B13:P16,PCLB003_PC3_24H_X3_B13:P17,PCLB003_PC3_24H_X3_B13:P18,PCLB003_PC3_24H_X3_B13:P19,PCLB003_PC3_24H_X3_B13:P20,PCLB003_PC3_24H_X3_B13:P21,PCLB003_PC3_24H_X3_B13:P22,PCLB003_PC3_24H_X3_B13:P23,PCLB003_PC3_24H_X3_B13:P24
5720,0.8509,0.6964,0.7763,-0.7216,-0.5528,-0.8905,1.1575,-10.4761,-0.2220,-0.4631,...,-2.7870,-0.9472,-1.2010,0.1184,2.1614,0.4863,-0.0174,1.3497,0.7177,0.6071
466,0.0117,-1.1202,-1.2383,0.2762,-1.9274,-1.1556,1.6463,2.9997,-0.1231,-0.9180,...,4.7663,4.8015,1.3907,0.8861,-2.5416,4.4342,-1.2144,4.7777,0.6917,0.2969


# Load meta data

In [17]:
loaded_pert_txt_GSE70138=data_load_module.Utils_Data_Load.load_txt(BASE_DIR,TXT_META_PATH_GSE70138)
loaded_pert_txt_GSE92742=data_load_module.Utils_Data_Load.load_txt(BASE_DIR,TXT_META_PATH_GSE92742)
loaded_pert_actual_txt_GSE70138=data_load_module.Utils_Data_Load.load_txt(BASE_DIR,PERT_META_PATH_GSE70138)
loaded_pert_actual_txt_GSE92742=data_load_module.Utils_Data_Load.load_txt(BASE_DIR,PERT_META_PATH_GSE92742)
loaded_landmark_txt=data_load_module.Utils_Data_Load.load_txt(BASE_DIR,TXT_LANDMARK_GENES_PATH)

# left join "loaded_pert_txt" and "loaded_pert_actual_txt"

In [10]:
set(loaded_pert_txt.columns).intersection(set(loaded_pert_actual_txt.columns))

NameError: name 'loaded_pert_txt' is not defined

In [18]:
joint_data_pert_pert_actual_GSE70138=loaded_pert_txt_GSE70138.merge(loaded_pert_actual_txt_GSE70138.drop(columns=['pert_iname', 'pert_type']),left_on='pert_id',right_on='pert_id',how='left')
joint_data_pert_pert_actual_GSE92742=loaded_pert_txt_GSE92742.merge(loaded_pert_actual_txt_GSE92742.drop(columns=['pert_iname', 'pert_type']),left_on='pert_id',right_on='pert_id',how='left')

In [15]:
display(joint_data_pert_pert_actual_GSE70138.head(2))
display(joint_data_pert_pert_actual_GSE92742.head(2))

,inst_id,rna_plate,rna_well,pert_id,pert_iname,pert_type,pert_dose,pert_dose_unit,pert_time,pert_time_unit,cell_id,is_touchstone,inchi_key_prefix,inchi_key,canonical_smiles,pubchem_cid
0,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:F13,ASG001_MCF7_24H_X1,F13,DMSO,DMSO,ctl_vehicle,0.1,%,24,h,MCF7,1.0,IAZDPXIOMUYVGZ,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,CS(=O)C,-666
1,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:G13,ASG001_MCF7_24H_X1,G13,DMSO,DMSO,ctl_vehicle,0.1,%,24,h,MCF7,1.0,IAZDPXIOMUYVGZ,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,CS(=O)C,-666


# Set index data

In [19]:
joint_data_pert_pert_actual_GSE70138.index=list(joint_data_pert_pert_actual_GSE70138['inst_id'])
joint_data_pert_pert_actual_GSE92742.index=list(joint_data_pert_pert_actual_GSE92742['inst_id'])
loaded_landmark_txt.index=[str(i) for i in loaded_landmark_txt['pr_gene_id']]

In [ ]:
display(joint_data_pert_pert_actual_GSE70138.head(2))
display(joint_data_pert_pert_actual_GSE92742.head(2))
display(loaded_landmark_txt.head(2))

# Remove -666 rows (non drugs)

In [20]:
joint_data_pert_pert_actual_remove_non_drug_GSE70138=joint_data_pert_pert_actual_GSE70138.drop(joint_data_pert_pert_actual_GSE70138[(joint_data_pert_pert_actual_GSE70138['canonical_smiles']=="-666")].index)
joint_data_pert_pert_actual_remove_non_drug_GSE92742=joint_data_pert_pert_actual_GSE92742.drop(joint_data_pert_pert_actual_GSE92742[(joint_data_pert_pert_actual_GSE92742['canonical_smiles']=="-666")].index)

In [22]:
display(joint_data_pert_pert_actual_remove_non_drug_GSE70138.head(2))
display(joint_data_pert_pert_actual_remove_non_drug_GSE92742.head(2))

NameError: name 'joint_data_pert_pert_actual_remove_non_drug' is not defined

# Leave only trt_cp in pert_type

In [21]:
# https://clue.io/connectopedia/perturbagen_types_and_controls
joint_data_pert_pert_actual_remove_non_drug_GSE70138=joint_data_pert_pert_actual_remove_non_drug_GSE70138[joint_data_pert_pert_actual_remove_non_drug_GSE70138['pert_type']=="trt_cp"]
joint_data_pert_pert_actual_remove_non_drug_GSE92742=joint_data_pert_pert_actual_remove_non_drug_GSE92742[joint_data_pert_pert_actual_remove_non_drug_GSE92742['pert_type']=="trt_cp"]

In [23]:
display(joint_data_pert_pert_actual_remove_non_drug_GSE70138.head())
print(joint_data_pert_pert_actual_remove_non_drug_GSE70138.shape)
display(joint_data_pert_pert_actual_remove_non_drug_GSE92742.head())
print(joint_data_pert_pert_actual_remove_non_drug_GSE92742.shape)

,inst_id,rna_plate,rna_well,pert_id,pert_iname,pert_type,pert_dose,pert_dose_unit,pert_time,pert_time_unit,cell_id,is_touchstone,inchi_key_prefix,inchi_key,canonical_smiles,pubchem_cid
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:A06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:A06,ASG001_MCF7_24H_X1,A06,BRD-A60070924,alpha-estradiol,trt_cp,10.00,um,24,h,MCF7,1.0,VOXZDWNPVJITMN,VOXZDWNPVJITMN-UHFFFAOYSA-N,CC12CCC3C(CCc4cc(O)ccc34)C1CCC2O,450
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:B06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:B06,ASG001_MCF7_24H_X1,B06,BRD-A60070924,alpha-estradiol,trt_cp,2.00,um,24,h,MCF7,1.0,VOXZDWNPVJITMN,VOXZDWNPVJITMN-UHFFFAOYSA-N,CC12CCC3C(CCc4cc(O)ccc34)C1CCC2O,450
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:C06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:C06,ASG001_MCF7_24H_X1,C06,BRD-A60070924,alpha-estradiol,trt_cp,0.40,um,24,h,MCF7,1.0,VOXZDWNPVJITMN,VOXZDWNPVJITMN-UHFFFAOYSA-N,CC12CCC3C(CCc4cc(O)ccc34)C1CCC2O,450
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:D06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:D06,ASG001_MCF7_24H_X1,D06,BRD-A60070924,alpha-estradiol,trt_cp,0.08,um,24,h,MCF7,1.0,VOXZDWNPVJITMN,VOXZDWNPVJITMN-UHFFFAOYSA-N,CC12CCC3C(CCc4cc(O)ccc34)C1CCC2O,450
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:E06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:E06,ASG001_MCF7_24H_X1,E06,BRD-A60070924,alpha-estradiol,trt_cp,10.00,um,24,h,MCF7,1.0,VOXZDWNPVJITMN,VOXZDWNPVJITMN-UHFFFAOYSA-N,CC12CCC3C(CCc4cc(O)ccc34)C1CCC2O,450


(663874, 16)


# Merge "loaded_gctx" and "loaded_landmark_txt" (leave only landmark genes)

In [22]:
expression_df_by_landmarks_GSE70138=pd.merge(loaded_gctx_GSE70138,loaded_landmark_txt,left_index=True,right_index=True,how='inner')
expression_df_by_landmarks_GSE92742=pd.merge(loaded_gctx_GSE92742,loaded_landmark_txt,left_index=True,right_index=True,how='inner')

In [26]:
display(expression_df_by_landmarks_GSE70138.head(2))
display(expression_df_by_landmarks_GSE92742.head(2))

,CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,CPC005_A375_6H_X3_B3_DUO52HI53LO:K06,CPC005_A375_6H_X1_B3_DUO52HI53LO:C19,CPC005_A375_6H_X2_B3_DUO52HI53LO:C19,CPC005_A375_6H_X3_B3_DUO52HI53LO:C19,CPC004_A375_6H_X1_B3_DUO52HI53LO:K13,CPC004_A375_6H_X2_B3_DUO52HI53LO:K13,CPC004_A375_6H_X3_B3_DUO52HI53LO:K13,CPC005_A375_6H_X1_B3_DUO52HI53LO:K20,...,PCLB003_PC3_24H_X3_B13:P18,PCLB003_PC3_24H_X3_B13:P19,PCLB003_PC3_24H_X3_B13:P20,PCLB003_PC3_24H_X3_B13:P21,PCLB003_PC3_24H_X3_B13:P22,PCLB003_PC3_24H_X3_B13:P23,PCLB003_PC3_24H_X3_B13:P24,pr_gene_id,pr_gene_symbol,pr_gene_title
55847,0.2106,1.1142,-0.5801,-0.9112,0.5033,3.0285,0.9118,0.0184,-0.0293,-1.2838,...,0.1329,0.4067,0.5962,-0.1673,1.3051,0.7453,0.1329,55847,CISD1,CDGSH iron sulfur domain 1
25803,0.2892,0.1883,0.6909,1.4429,6.0724,-1.2484,0.8752,3.4359,0.1169,-1.3850,...,-1.9368,-1.1910,0.1538,-0.9033,-0.9033,-1.6760,0.5652,25803,SPDEF,SAM pointed domain containing ETS transcriptio...


# Transpose df

In [ ]:
expression_df_by_landmarks_T_GSE70138=expression_df_by_landmarks_GSE70138.T
expression_df_by_landmarks_T_GSE92742=expression_df_by_landmarks_GSE92742.T

In [28]:
display(expression_df_by_landmarks_T_GSE70138.head(2))
display(expression_df_by_landmarks_T_GSE92742.head(2))

,55847,25803,3480,2946,9517,25987,23670,11182,2146,3385,...,5526,25836,5523,1837,1955,22930,7840,3046,55278,4122
CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,0.2106,0.2892,-0.3222,-0.5751,-0.8432,-0.0981,0.1181,0.3545,-0.9332,0.9282,...,-0.2673,0.5202,0.4783,0.3221,1.7838,0.6258,-0.6607,1.5576,0.5938,0.8408
CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,1.1142,0.1883,-0.0239,1.121,-0.8318,-0.4604,-0.7833,0.2543,-0.8127,-0.2484,...,-0.1253,-0.4552,1.8226,1.9999,1.1418,-1.5645,1.2444,1.9761,0.52,1.0644


# Give gene names to columns

In [ ]:
expression_df_by_landmarks_T_GSE70138.columns=list(expression_df_by_landmarks_T_GSE70138.loc["pr_gene_symbol",:])
expression_df_by_landmarks_T_GSE92742.columns=list(expression_df_by_landmarks_T_GSE92742.loc["pr_gene_symbol",:])

In [30]:
display(expression_df_by_landmarks_T_GSE70138.head(2))
display(expression_df_by_landmarks_T_GSE92742.head(2))

,CISD1,SPDEF,IGF1R,GSTM2,SPTLC2,TSKU,TMEM2,SLC2A6,EZH2,ICAM3,...,PPP2R5B,NIPBL,PPP2R3A,DTNA,MEGF9,RAB3GAP1,ALMS1,HBE1,QRSL1,MAN2A2
CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,0.2106,0.2892,-0.3222,-0.5751,-0.8432,-0.0981,0.1181,0.3545,-0.9332,0.9282,...,-0.2673,0.5202,0.4783,0.3221,1.7838,0.6258,-0.6607,1.5576,0.5938,0.8408
CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,1.1142,0.1883,-0.0239,1.121,-0.8318,-0.4604,-0.7833,0.2543,-0.8127,-0.2484,...,-0.1253,-0.4552,1.8226,1.9999,1.1418,-1.5645,1.2444,1.9761,0.52,1.0644


In [ ]:
test_keys=[str(one) for one in list(loaded_landmark_txt['pr_gene_id'])]
test_values=loaded_landmark_txt['pr_gene_symbol']
res = {test_keys[i]: test_values[i] for i in range(len(test_keys))}
res

# Remove unnecessary data (last 3 rows)

In [ ]:
expression_df_by_landmarks_T_wo_unnecessary_rows_GSE70138=expression_df_by_landmarks_T_GSE70138.drop(["pr_gene_id","pr_gene_symbol","pr_gene_title"],axis='index')
expression_df_by_landmarks_T_wo_unnecessary_rows_GSE92742=expression_df_by_landmarks_T_GSE92742.drop(["pr_gene_id","pr_gene_symbol","pr_gene_title"],axis='index')

In [ ]:
display(expression_df_by_landmarks_T_wo_unnecessary_rows_GSE70138.head(2))
display(expression_df_by_landmarks_T_wo_unnecessary_rows_GSE92742.head(2))

# Merge "expression_df_by_landmarks_T_wo_unnecessary_rows" and "joint_data_pert_pert_actual_remove_non_drug"

In [ ]:
display(expression_df_by_landmarks_T_wo_unnecessary_rows_GSE70138)
display(joint_data_pert_pert_actual_remove_non_drug_GSE70138)
display(expression_df_by_landmarks_T_wo_unnecessary_rows_GSE92742)
display(joint_data_pert_pert_actual_remove_non_drug_GSE92742)

In [ ]:
expression_pert_df_GSE70138=pd.merge(expression_df_by_landmarks_T_wo_unnecessary_rows_GSE70138,joint_data_pert_pert_actual_remove_non_drug_GSE70138[['pert_iname']],left_index=True,right_index=True,how='inner')
expression_pert_df_GSE92742=pd.merge(expression_df_by_landmarks_T_wo_unnecessary_rows_GSE92742,joint_data_pert_pert_actual_remove_non_drug_GSE92742[['pert_iname']],left_index=True,right_index=True,how='inner')

In [38]:
display(expression_pert_df_GSE70138.head(2))
display(expression_pert_df_GSE92742.head(2))

,CISD1,SPDEF,IGF1R,GSTM2,SPTLC2,TSKU,TMEM2,SLC2A6,EZH2,ICAM3,...,NIPBL,PPP2R3A,DTNA,MEGF9,RAB3GAP1,ALMS1,HBE1,QRSL1,MAN2A2,pert_iname
CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,0.2106,0.2892,-0.3222,-0.5751,-0.8432,-0.0981,0.1181,0.3545,-0.9332,0.9282,...,0.5202,0.4783,0.3221,1.7838,0.6258,-0.6607,1.5576,0.5938,0.8408,quinpirole
CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,1.1142,0.1883,-0.0239,1.121,-0.8318,-0.4604,-0.7833,0.2543,-0.8127,-0.2484,...,-0.4552,1.8226,1.9999,1.1418,-1.5645,1.2444,1.9761,0.52,1.0644,quinpirole


In [39]:
display(expression_pert_df_GSE70138.shape)
display(expression_pert_df_GSE92742.shape)

(663874, 978)

# Concatenate two LINCS data

In [ ]:
expression_pert_df_GSE70138['LINCS']=["GSE70138"]*expression_pert_df_GSE70138.shape[0]
expression_pert_df_GSE92742['LINCS']=["GSE92742"]*expression_pert_df_GSE92742.shape[0]

In [29]:
expression_pert_df_cat=pd.concat((expression_pert_df_GSE70138,expression_pert_df_GSE92742),axis=0)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [ ]:
expression_pert_df_cat

# Save data as csv

In [26]:
entire_compounds=list(expression_pert_df_cat['pert_iname'].unique())
entire_compounds.sort()
entire_compounds

['(+)-3-(1-propyl-piperidin-3-yl)-phenol',
 '(+/-)-7-hydroxy-2-(N,N-di-n-propylamino)tetralin',
 '1,2,3,4,5,6-hexabromocyclohexane',
 '1,2,3,4-tetrahydroisoquinoline',
 '1,2-dichlorobenzene',
 '1,2-propylene-glycol',
 '1-benzylimidazole',
 '1-methylisoquinoline',
 '1-monopalmitin',
 '1-phenylbiguanide',
 '10-DEBC',
 '10-hydroxycamptothecin',
 '10H-phenothiazin-10-yl)(p-tolyl)methanone',
 '1271738-62-5',
 '15-delta-prostaglandin-j2',
 '16,16-dimethylprostaglandin-e2',
 '17-beta-estradiol',
 '17-hydroxyprogesterone-caproate',
 '1B Parent',
 "2',5'-dideoxyadenosine",
 '2-(4-methoxybenzylthio)-6-methylpyrimidin-4-ol',
 '2-(biphenyl-4-ylsulfonamido)pentanedioic-acid',
 '2-aminobenzenesulfonamide',
 '2-aminopurine',
 '2-aminotetralin',
 '2-fluoropalmitic-acid',
 '2-iminobiotin',
 '2-methoxyestradiol',
 "3,3'-diindolylmethane",
 '3,6-dimethoxyflavone',
 '3-amino-benzamide',
 '3-deazaadenosine',
 '3-matida',
 '3-methyl-GABA',
 '4,5,6,7-tetrabromobenzotriazole',
 '4-(2-amino-ethyl)-benzenesulfo

In [27]:
d={'entire_compounds':entire_compounds}
df=pd.DataFrame(d)
df

,entire_compounds
0,(+)-3-(1-propyl-piperidin-3-yl)-phenol
1,"(+/-)-7-hydroxy-2-(N,N-di-n-propylamino)tetralin"
2,"1,2,3,4,5,6-hexabromocyclohexane"
3,"1,2,3,4-tetrahydroisoquinoline"
4,"1,2-dichlorobenzene"
...,...
20497,zonisamide
20498,zopiclone
20499,zosuquidar
20500,zoxazolamine


In [ ]:
fn='./entire_compounds.csv'
df.to_csv(fn,sep=',',encoding='utf-8',index=False)

In [ ]:
BRD_compounds=expression_pert_df[expression_pert_df['pert_iname'].str.contains('BRD-')]

In [ ]:
BRD_compounds=BRD_compounds['pert_iname'].value_counts().to_frame().reset_index()
BRD_compounds.columns=['compounds','freq']
fn='./BRD_compounds_and_their_frequency.csv'
BRD_compounds.to_csv(fn,sep=',',encoding='utf-8',index=False)

# Select frequent compounds (freq>100, 1170 compounds)

In [30]:
pert_counts_df=expression_pert_df_cat['pert_iname'].value_counts().to_frame()
pert_counts_df

,pert_iname
bortezomib,6290
MG-132,6289
vorinostat,4768
geldanamycin,3907
trichostatin-a,3692
...,...
BRD-K80564983,2
BRD-K10196668,1
BRD-K47471281,1
BRD-K82494568,1


In [31]:
criterion_num=40
selected_compounds=list(pert_counts_df[pert_counts_df['pert_iname']>criterion_num].index)    ;display(selected_compounds);len(selected_compounds)

['bortezomib',
 'MG-132',
 'vorinostat',
 'geldanamycin',
 'trichostatin-a',
 'wortmannin',
 'GSK-1059615',
 'PD-0325901',
 'dasatinib',
 'sirolimus',
 'withaferin-a',
 'CGP-60474',
 'curcumin',
 'olaparib',
 'tozasertib',
 'mitoxantrone',
 'sulforaphane',
 'fulvestrant',
 'genistein',
 'estradiol',
 'veliparib',
 'BI-2536',
 'saracatinib',
 'radicicol',
 'tamoxifen',
 'AZD-7762',
 'LY-294002',
 'resveratrol',
 'BRD-K98948170',
 'selumetinib',
 'sorafenib',
 'raloxifene',
 'entinostat',
 'PF-04217903',
 'gefitinib',
 'vemurafenib',
 'AS-605240',
 'GSK-1070916',
 'imatinib',
 'tert-butylhydroquinone',
 'BRD-K93367411',
 'MLN-8054',
 'alvocidib',
 'barasertib-HQPA',
 'fostamatinib',
 'MK-2206',
 'tivozanib',
 'dexamethasone',
 'PI-103',
 'GW-843682X',
 'crizotinib',
 'lapatinib',
 'nilotinib',
 'tanespimycin',
 'Y-27632',
 'GDC-0941',
 'neratinib',
 'PD-173074',
 'SJB-shh-31',
 'thioridazine',
 'palbociclib',
 'PLX-4720',
 'AKT-inhibitor-1-2',
 'MK-1775',
 'torin-1',
 'PD-184352',
 'NVP-

5883

# Save data as CSV

In [71]:
filtered_compounds=list(pert_counts_df[pert_counts_df['pert_iname']>criterion_num].index)
d={'compounds':filtered_compounds}
df=pd.DataFrame(d)
fn='./selected_compounds_two_lincs_freq{}_len{}.csv'.format(criterion_num,len(filtered_compounds))
df.to_csv(fn,sep=',',encoding='utf-8',index=False)

# Leave samples associated with selected compounds

In [32]:
expression_freq_pert_df=expression_pert_df_cat[expression_pert_df_cat['pert_iname'].isin(selected_compounds)]
expression_freq_pert_df

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,CABIN1,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1,pert_iname,LINCS
REP.A001_A375_24H_X1_B22:A07,0.4163,-0.2225,0.9273,0.0202,0.0,-0.4607,0.0204,0.6745,-0.3362,0.6315,...,-0.7345,3.5454,-7.9314,-1.4845,0.7707,0.2316,-0.8891,-0.1372,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A08,-0.1698,-0.1234,-0.1034,0.464,-16.6213,-0.4256,0.751,1.345,-0.4169,-1.3208,...,-0.0553,1.4655,0.342,-2.3748,-0.4059,0.0063,-0.4077,13.4469,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A09,-0.3789,-0.3441,0.3622,-4.0071,0.0,0.0,0.8806,0.0,0.2052,-1.216,...,0.926,0.3322,0.1071,-0.0438,0.8398,-0.9396,1.5787,12.1437,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A10,0.0907,-0.9418,-0.0818,-1.0441,0.0,-0.3292,1.5043,0.6745,-0.1755,0.252,...,0.517,0.4509,-0.5566,0.2643,-0.3026,0.816,0.2214,12.1766,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A11,0.5457,1.2708,0.1439,0.0958,0.0,0.9065,-0.7551,0.487,0.2947,0.4657,...,-0.6222,1.3378,-0.0955,-2.066,0.2742,-1.6494,-0.8301,13.1083,aminoguanidine,GSE70138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.3662,-2.1044,0.2024,0.8759,-0.9267,-0.0539,-0.1612,2.8925,0.372,-0.0475,...,-0.976,0.0881,-2.5475,0.4033,1.1583,0.6428,-1.2056,-0.2261,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P21,0.3775,-0.4531,-0.0109,-0.266,-0.1719,0.0985,-0.1459,-2.066,-0.375,-1.2346,...,-1.0873,-2.0991,-1.4825,2.1151,0.0,0.5136,1.1579,-0.087,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P22,0.5546,0.0,-0.3511,1.6628,-0.8406,-0.6109,-0.4939,-1.4338,0.0589,0.0729,...,-1.0419,-0.2673,-1.2113,-1.0933,1.1583,-0.3659,-0.1505,0.4603,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P23,0.3037,-0.666,1.176,0.5265,-1.0765,0.36,0.4682,7.2672,-0.4834,-0.5445,...,-0.9612,-2.7238,-0.7698,0.3769,0.2699,0.2177,-1.0949,0.3538,wortmannin,GSE92742


# Remove BRDs

In [33]:
expression_freq_pert_df=expression_freq_pert_df[~expression_freq_pert_df['pert_iname'].str.contains('BRD-')]

In [34]:
expression_freq_pert_df['pert_iname'].value_counts()

bortezomib                6290
MG-132                    6289
vorinostat                4768
geldanamycin              3907
trichostatin-a            3692
                          ... 
5-nonyloxytryptamine        42
SA-1920013                  41
hydrocortisone-acetate      41
VU-0413255-1                41
TL-HRAS-24                  41
Name: pert_iname, Length: 3820, dtype: int64

# Convert pert_iname to number

In [35]:
keys = list(expression_freq_pert_df['pert_iname'].unique())
values = list(range(len(expression_freq_pert_df['pert_iname'].unique())))
dictionary = dict(zip(keys, values))

In [29]:
print(dictionary)

{'aminoguanidine': 0, 'pritelivir': 1, 'batimastat': 2, 'CX-516': 3, 'aminoglutethimide': 4, 'oxaprozin': 5, 'physostigmine': 6, 'blonanserin': 7, 'zaldaride': 8, 'bortezomib': 9, 'josamycin': 10, 'licochalcone-a': 11, 'clobetasol': 12, 'propafenone': 13, 'PD-153035': 14, 'imiquimod': 15, 'saquinavir': 16, 'nicardipine': 17, 'thiocolchicoside': 18, 'tegobuvir': 19, 'VX-765': 20, 'lestaurtinib': 21, 'ASA-404': 22, 'proguanil': 23, 'tolperisone': 24, 'lomeguatrib': 25, 'cyproheptadine': 26, 'ganaxolone': 27, 'PKI-179': 28, 'selegiline': 29, 'calcitriol': 30, 'ramelteon': 31, 'benzydamine': 32, 'pyrazinamide': 33, 'inosine': 34, 'ZD-7114': 35, 'MG-132': 36, 'PCA-4248': 37, 'edoxaban': 38, 'dutasteride': 39, 'acebutolol': 40, 'potassium-canrenoate': 41, 'ebselen': 42, 'ziprasidone': 43, 'topiramate': 44, 'hypericin': 45, 'disopyramide': 46, 'anisodamine': 47, 'MK-8245': 48, '17-hydroxyprogesterone-caproate': 49, 'ostarine': 50, 'dimesna': 51, 'timolol': 52, 'torasemide': 53, 'amikacin': 54

In [36]:
number_of_perturbagens=len(list(expression_freq_pert_df['pert_iname'].unique()))
configurable_parameters_dict['number_of_perturbagens']=number_of_perturbagens
number_of_perturbagens

3820

In [31]:
expression_freq_pert_df

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,CABIN1,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1,pert_iname,LINCS
REP.A001_A375_24H_X1_B22:A07,0.4163,-0.2225,0.9273,0.0202,0.0,-0.4607,0.0204,0.6745,-0.3362,0.6315,...,-0.7345,3.5454,-7.9314,-1.4845,0.7707,0.2316,-0.8891,-0.1372,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A08,-0.1698,-0.1234,-0.1034,0.464,-16.6213,-0.4256,0.751,1.345,-0.4169,-1.3208,...,-0.0553,1.4655,0.342,-2.3748,-0.4059,0.0063,-0.4077,13.4469,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A09,-0.3789,-0.3441,0.3622,-4.0071,0.0,0.0,0.8806,0.0,0.2052,-1.216,...,0.926,0.3322,0.1071,-0.0438,0.8398,-0.9396,1.5787,12.1437,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A10,0.0907,-0.9418,-0.0818,-1.0441,0.0,-0.3292,1.5043,0.6745,-0.1755,0.252,...,0.517,0.4509,-0.5566,0.2643,-0.3026,0.816,0.2214,12.1766,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A11,0.5457,1.2708,0.1439,0.0958,0.0,0.9065,-0.7551,0.487,0.2947,0.4657,...,-0.6222,1.3378,-0.0955,-2.066,0.2742,-1.6494,-0.8301,13.1083,aminoguanidine,GSE70138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.3662,-2.1044,0.2024,0.8759,-0.9267,-0.0539,-0.1612,2.8925,0.372,-0.0475,...,-0.976,0.0881,-2.5475,0.4033,1.1583,0.6428,-1.2056,-0.2261,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P21,0.3775,-0.4531,-0.0109,-0.266,-0.1719,0.0985,-0.1459,-2.066,-0.375,-1.2346,...,-1.0873,-2.0991,-1.4825,2.1151,0.0,0.5136,1.1579,-0.087,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P22,0.5546,0.0,-0.3511,1.6628,-0.8406,-0.6109,-0.4939,-1.4338,0.0589,0.0729,...,-1.0419,-0.2673,-1.2113,-1.0933,1.1583,-0.3659,-0.1505,0.4603,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P23,0.3037,-0.666,1.176,0.5265,-1.0765,0.36,0.4682,7.2672,-0.4834,-0.5445,...,-0.9612,-2.7238,-0.7698,0.3769,0.2699,0.2177,-1.0949,0.3538,wortmannin,GSE92742


In [37]:
pert_iname_num=expression_freq_pert_df['pert_iname'].map(dictionary)

In [38]:
expression_freq_pert_df['pert_iname_num']=pert_iname_num

In [34]:
expression_freq_pert_df

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1,pert_iname,LINCS,pert_iname_num
REP.A001_A375_24H_X1_B22:A07,0.4163,-0.2225,0.9273,0.0202,0.0,-0.4607,0.0204,0.6745,-0.3362,0.6315,...,3.5454,-7.9314,-1.4845,0.7707,0.2316,-0.8891,-0.1372,aminoguanidine,GSE70138,0
REP.A001_A375_24H_X1_B22:A08,-0.1698,-0.1234,-0.1034,0.464,-16.6213,-0.4256,0.751,1.345,-0.4169,-1.3208,...,1.4655,0.342,-2.3748,-0.4059,0.0063,-0.4077,13.4469,aminoguanidine,GSE70138,0
REP.A001_A375_24H_X1_B22:A09,-0.3789,-0.3441,0.3622,-4.0071,0.0,0.0,0.8806,0.0,0.2052,-1.216,...,0.3322,0.1071,-0.0438,0.8398,-0.9396,1.5787,12.1437,aminoguanidine,GSE70138,0
REP.A001_A375_24H_X1_B22:A10,0.0907,-0.9418,-0.0818,-1.0441,0.0,-0.3292,1.5043,0.6745,-0.1755,0.252,...,0.4509,-0.5566,0.2643,-0.3026,0.816,0.2214,12.1766,aminoguanidine,GSE70138,0
REP.A001_A375_24H_X1_B22:A11,0.5457,1.2708,0.1439,0.0958,0.0,0.9065,-0.7551,0.487,0.2947,0.4657,...,1.3378,-0.0955,-2.066,0.2742,-1.6494,-0.8301,13.1083,aminoguanidine,GSE70138,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.3662,-2.1044,0.2024,0.8759,-0.9267,-0.0539,-0.1612,2.8925,0.372,-0.0475,...,0.0881,-2.5475,0.4033,1.1583,0.6428,-1.2056,-0.2261,wortmannin,GSE92742,2175
PCLB003_PC3_24H_X3_B13:P21,0.3775,-0.4531,-0.0109,-0.266,-0.1719,0.0985,-0.1459,-2.066,-0.375,-1.2346,...,-2.0991,-1.4825,2.1151,0.0,0.5136,1.1579,-0.087,wortmannin,GSE92742,2175
PCLB003_PC3_24H_X3_B13:P22,0.5546,0.0,-0.3511,1.6628,-0.8406,-0.6109,-0.4939,-1.4338,0.0589,0.0729,...,-0.2673,-1.2113,-1.0933,1.1583,-0.3659,-0.1505,0.4603,wortmannin,GSE92742,2175
PCLB003_PC3_24H_X3_B13:P23,0.3037,-0.666,1.176,0.5265,-1.0765,0.36,0.4682,7.2672,-0.4834,-0.5445,...,-2.7238,-0.7698,0.3769,0.2699,0.2177,-1.0949,0.3538,wortmannin,GSE92742,2175


# Make train data

In [39]:
before_standardization_GSE70138=expression_freq_pert_df[expression_freq_pert_df['LINCS']=='GSE70138']
before_standardization_GSE92742=expression_freq_pert_df[expression_freq_pert_df['LINCS']=='GSE92742']

In [40]:
X_data_GSE70138=np.array(before_standardization_GSE70138.iloc[:,:-3], dtype=np.float)
y_data_GSE70138=np.array(before_standardization_GSE70138.iloc[:,-1], dtype=np.int)
X_data_GSE92742=np.array(before_standardization_GSE92742.iloc[:,:-3], dtype=np.float)
y_data_GSE92742=np.array(before_standardization_GSE92742.iloc[:,-1], dtype=np.int)

# X_data=np.array(expression_freq_pert_df.iloc[:,:-3], dtype=np.float)
# y_data=np.array(expression_freq_pert_df.iloc[:,-1], dtype=np.int)

# Perform standardization

In [41]:
# only standardization separately
standard_scaler_GSE70138=StandardScaler()
X_data_standardized_GSE70138=standard_scaler_GSE70138.fit_transform(X_data_GSE70138)
standard_scaler_GSE92742=StandardScaler()
X_data_standardized_GSE92742=standard_scaler_GSE92742.fit_transform(X_data_GSE92742)

# standardization -> min max normalization
# minmax_scaler_GSE70138=MinMaxScaler(feature_range=(-1, 1))
# X_data_minmax_GSE70138=minmax_scaler_GSE70138.fit_transform(X_data_standardized_GSE70138)
# minmax_scaler_GSE92742=MinMaxScaler(feature_range=(-1, 1))
# X_data_minmax_GSE92742=minmax_scaler_GSE92742.fit_transform(X_data_standardized_GSE92742)

# only min max normalization separately
# minmax_scaler_GSE70138=MinMaxScaler()
# X_data_minmax_GSE70138=minmax_scaler_GSE70138.fit_transform(X_data_GSE70138)
# minmax_scaler_GSE92742=MinMaxScaler()
# X_data_minmax_GSE92742=minmax_scaler_GSE92742.fit_transform(X_data_GSE92742)

# standard_scaler=StandardScaler()
# X_data_standardized=standard_scaler.fit_transform(X_data)

# Concat X and y again

In [42]:
expression_pert_np_copied_GSE70138=np.concatenate((X_data_standardized_GSE70138, y_data_GSE70138[:,np.newaxis]), axis=1)
expression_pert_np_copied_GSE92742=np.concatenate((X_data_standardized_GSE92742, y_data_GSE92742[:,np.newaxis]), axis=1)

# expression_pert_np_copied_GSE70138=np.concatenate((X_data_minmax_GSE70138, y_data_GSE70138[:,np.newaxis]), axis=1)
# expression_pert_np_copied_GSE92742=np.concatenate((X_data_minmax_GSE92742, y_data_GSE92742[:,np.newaxis]), axis=1)

# expression_pert_np_copied=np.concatenate((X_data, y_data[:,np.newaxis]), axis=1)

In [103]:
display(expression_pert_np_copied_GSE70138)
display(expression_pert_np_copied_GSE92742)

array([[ 1.37101204e-01, -2.06940671e-01,  5.39252562e-01, ...,
        -8.58415604e-01, -1.65031996e-01,  0.00000000e+00],
       [-2.21706549e-01, -1.59160321e-01, -1.93644298e-01, ...,
        -4.48487137e-01,  7.55646571e+00,  0.00000000e+00],
       [-3.49716613e-01, -2.65569239e-01,  1.37428545e-01, ...,
         1.24299991e+00,  6.81569811e+00,  0.00000000e+00],
       ...,
       [ 2.59788217e+00,  6.95003915e-01,  2.01023716e+00, ...,
        -1.57177362e-01,  2.80389140e+00,  1.56900000e+03],
       [ 4.40076725e-01,  7.09902129e-01, -1.06911011e+00, ...,
        -9.49785186e-01,  6.37863698e-01,  1.34400000e+03],
       [-9.27200551e-01, -1.26181751e-01,  1.63970005e+00, ...,
         5.59133033e-01,  7.27276432e-01,  3.11000000e+02]])

array([[-6.91922196e-01, -4.71341493e-01, -5.23085028e-01, ...,
        -3.18820760e-01,  8.75690331e-01,  1.74800000e+03],
       [-9.79655006e-01, -1.34656755e-01, -3.23063600e+00, ...,
        -1.98186023e-01,  5.44638769e-02,  1.74800000e+03],
       [-1.37593729e-01, -5.01900883e-01, -2.07739660e-01, ...,
        -9.48669997e-01,  3.08148275e-01,  1.74800000e+03],
       ...,
       [ 2.54549546e-01, -9.46372439e-02, -2.30285723e-01, ...,
        -2.19594436e-01,  3.18372573e-01,  2.17500000e+03],
       [ 8.61909765e-02, -4.48594675e-01,  9.13571616e-01, ...,
        -1.02190038e+00,  2.68423620e-01,  2.17500000e+03],
       [ 5.46912353e-01, -6.77604055e-01,  1.53147297e-01, ...,
        -1.22237110e-01,  1.30356291e+00,  2.17500000e+03]])

# Data concatenation

In [ ]:
expression_pert_np_copied=np.concatenate((expression_pert_np_copied_GSE70138,expression_pert_np_copied_GSE92742),axis=0)

# Data shape check

In [ ]:
expression_pert_np_copied.shape

# For fast test

In [ ]:
fast_test_number=10000

In [ ]:
expression_pert_np_copied_small=expression_pert_np_copied[:fast_test_number,:]

In [ ]:
expression_pert_np_copied_small.shape

(10000, 978)

In [ ]:
configurable_parameters_dict['number_of_perturbagens']=len(np.unique(expression_pert_np_copied_small[:fast_test_number,-1]))
configurable_parameters_dict

{'BASE_DIR': '/workspace/datafiles/2023/04/02_cha_drug_repositioning/paper_data',
 'GCTX_PATH_GSE70138': 'GSE70138_Broad_LINCS_Level4_ZSPCINF_mlr12k_n345976x12328_2017-03-06.gctx',
 'GCTX_PATH_GSE92742': 'GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx',
 'TXT_META_PATH_GSE70138': 'GSE70138_Broad_LINCS_inst_info_2017-03-06.txt',
 'TXT_META_PATH_GSE92742': 'GSE92742_Broad_LINCS_inst_info.txt',
 'PERT_META_PATH_GSE70138': 'GSE70138_Broad_LINCS_pert_info_2017-03-06.txt',
 'PERT_META_PATH_GSE92742': 'GSE92742_Broad_LINCS_pert_info.txt',
 'number_of_perturbagens': 114}

In [ ]:
configurable_parameters_dict['number_of_perturbagens']=len(np.unique(expression_pert_np_copied[:,-1]))
configurable_parameters_dict

{'BASE_DIR': '/workspace/datafiles/2023/04/02_cha_drug_repositioning/paper_data',
 'GCTX_PATH_GSE70138': 'GSE70138_Broad_LINCS_Level4_ZSPCINF_mlr12k_n345976x12328_2017-03-06.gctx',
 'GCTX_PATH_GSE92742': 'GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx',
 'TXT_META_PATH_GSE70138': 'GSE70138_Broad_LINCS_inst_info_2017-03-06.txt',
 'TXT_META_PATH_GSE92742': 'GSE92742_Broad_LINCS_inst_info.txt',
 'PERT_META_PATH_GSE70138': 'GSE70138_Broad_LINCS_pert_info_2017-03-06.txt',
 'PERT_META_PATH_GSE92742': 'GSE92742_Broad_LINCS_pert_info.txt',
 'number_of_perturbagens': 3820}

In [ ]:
# y_train_test=expression_pert_np_copied_small[:fast_test_number,-1].astype(int)
y_train_test=expression_pert_np_copied[:,-1].astype(int)

class_counts = np.bincount(y_train_test)
n_samples = len(y_train_test)
total_samples = np.sum(class_counts)
class_weights = (1 / n_samples) * (total_samples / class_counts)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
configurable_parameters_dict['class_weight_dict']=class_weight_dict

In [ ]:
class_weight_dict

{0: 0.008064516129032258,
 1: 0.007936507936507938,
 2: 0.0034482758620689655,
 3: 0.007936507936507938,
 4: 0.0037735849056603774,
 5: 0.003424657534246575,
 6: 0.00411522633744856,
 7: 0.007936507936507938,
 8: 0.0053475935828877,
 9: 0.00015898251192368842,
 10: 0.007462686567164179,
 11: 0.007142857142857143,
 12: 0.0029761904761904765,
 13: 0.0034722222222222225,
 14: 0.008,
 15: 0.0032051282051282055,
 16: 0.004098360655737705,
 17: 0.002247191011235955,
 18: 0.005235602094240838,
 19: 0.008,
 20: 0.008064516129032258,
 21: 0.004854368932038835,
 22: 0.007936507936507938,
 23: 0.005154639175257732,
 24: 0.007936507936507938,
 25: 0.007936507936507938,
 26: 0.005494505494505495,
 27: 0.008,
 28: 0.008064516129032258,
 29: 0.003831417624521073,
 30: 0.0018552875695732839,
 31: 0.008064516129032258,
 32: 0.004878048780487805,
 33: 0.004524886877828055,
 34: 0.008,
 35: 0.005050505050505051,
 36: 0.0001590077913817777,
 37: 0.005263157894736842,
 38: 0.008130081300813009,
 39: 0.0079

# Main script

In [ ]:
torch.cuda.empty_cache()

In [ ]:
GET_LIST_OF_CORRECT_AND_INCORRECT_CASES_FOR_CAM=False
ROC_WITH_CI=False
MULTIPLE_MODEL_ROC_IN_ONE_PLOT=False
SUBGROUP_ROC_ANALYSIS=False
CI_IN_PERFORMANCES_IN_SUBGROUPS=True

In [51]:
if __name__=='__main__':
  # ================================================================================
  # Create objects
  
  param_super_class_obj=param_class_module.Param_Super_Class(configurable_parameters_dict)
  utils_data_processing_obj=data_processing_module.Utils_Data_Processing(configurable_parameters_dict)
  train_class_obj=train_module.Train_Class(configurable_parameters_dict)
  test_class_obj=test_model_module.Test_Class(configurable_parameters_dict)
  
  # ================================================================================
  # print(train_class_obj.train_start(expression_pert_np_copied_small))
  # print(train_class_obj.train_start(expression_pert_np_copied))
  
  # ================================================================================
  # predictions_df=test_class_obj.test_start(expression_pert_np_copied_small)
  predictions_df=test_class_obj.test_start(expression_pert_np_copied)
  # display(predictions_df)
 
  def l2_normalize(row):
      norm=np.linalg.norm(row)
      return row/norm

  predictions_df = predictions_df.apply(l2_normalize, axis=1)
  # print(predictions_df)
  
  # ================================================================================
  # Add drug label
  # predictions_df['drug']=expression_pert_np_copied_small[:,-1]
  predictions_df['drug']=expression_pert_np_copied[:,-1]
  # print('predictions_df',predictions_df)
  
  # ================================================================================
  d={'drug_num':list(dictionary.values()),
     'drug_string':list(dictionary.keys())}
  df=pd.DataFrame(d)
  
  finale_pred=predictions_df.merge(df,left_on='drug',right_on='drug_num',how='inner')
  print('finale_pred',finale_pred)
  
  with open('./result_files/embedding_vectors.pkl','wb') as f:
    pickle.dump(finale_pred,f)

finale_pred                0         1         2         3         4         5         6  \
0      -0.195774  0.208246 -0.200245 -0.187345  0.298707  0.040652  0.061265   
1      -0.206023  0.269305 -0.164176 -0.130092  0.278392  0.169211  0.038797   
2      -0.229680  0.280046 -0.207327 -0.124709  0.297977  0.150674  0.027637   
3      -0.180996  0.208216 -0.170820 -0.186510  0.247666  0.029806  0.105625   
4      -0.292073  0.245195 -0.254652 -0.015646  0.319038  0.075625  0.082516   
...          ...       ...       ...       ...       ...       ...       ...   
680904 -0.078206 -0.067493 -0.127657  0.138919  0.194810 -0.251248  0.218485   
680905 -0.116378  0.009478 -0.121621  0.184124  0.091733 -0.154005  0.251708   
680906 -0.182747  0.154614 -0.137723  0.249791  0.183348 -0.165592  0.280881   
680907 -0.214649  0.059514 -0.125397  0.211409  0.172681 -0.116643  0.209058   
680908 -0.138644  0.064946 -0.115373  0.216691  0.111257 -0.080148  0.197466   

               7         8 